# Template

This notebook allows users to find common and specific genes in their experiment of interest using an *existing* VAE model and selecting a template experiment that is included in the training compendium.

This notebook will generate a `generic_gene_summary_<experiment id>.tsv` file that contains z-scores per gene that indicates how specific a gene is the experiment in question.

In [1]:
%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2

In [2]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from ponyo import utils, simulate_expression_data
from sophie import (
    process,
    stats,
    ranking,
)
# Remove later
import pickle

/home/alexandra/anaconda3/envs/sophie/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
Using TensorFlow backend.


## Inputs

User needs to fill in the [config file](config_new_experiment.tsv) following the instructions provided in the [readme]](README.md)

In [3]:
# Read in config variables
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))

config_filename = "config_example.tsv"

params = utils.read_config(config_filename)

In [4]:
# Load config params

# Local directory to store intermediate files
local_dir = params["local_dir"]

# Normalized compendium filename
normalized_compendium_filename = params["normalized_compendium_filename"]

# Metadata file that maps experiment ids to sample ids
experiment_to_sample_filename = params["experiment_to_sample_filename"]

# Delimiter used in metadata file
metadata_delimiter = params["metadata_delimiter"]

# Column header corresponding to experiment id and sample id
experiment_id_colname = params["experiment_id_colname"]
sample_id_colname = params["sample_id_colname"]

# ID for template experiment to be selected
project_id = params["project_id"]

# Number of simulated experiments to generate
num_runs = params["num_simulated"]

# Directory containing trained VAE model
vae_model_dir = params["vae_model_dir"]

# Size of the latent dimension
latent_dim = params["latent_dim"]

# Scaler transform used to scale compendium data into 0-1 range for training
scaler_filename = params["scaler_filename"]

# Which DE method to use
# We recommend that if data is RNA-seq then use DESeq2
# If data is microarray then use Limma
de_method = params["DE_method"]

# If using DE-seq, setting this parameter will
# remove genes below a certain threshold
count_threshold = params["count_threshold"]

# Metadata file that specifies which samples to keep for DE analysis (Optional)
template_process_samples_filename = params["template_process_samples_filename"]
    
# Metadata file that specifies sample grouping for DE analysis
template_DE_grouping_filename = params["template_DE_grouping_filename"]

# Statistic to use to rank genes or pathways by
# Choices are {} FILL IN
col_to_rank_genes = params["rank_genes_by"]

In [5]:
# Files generated by this notebook

# File storing normalized template experiment
normalized_template_filename = params["normalized_template_filename"]

# File storing un-normalized template experiment for us in downstream DE analysis
raw_template_filename = params["raw_template_filename"]

# File storing processed template experiment,
# after samples have been selected for comparison in DE analysis
processed_template_filename = params["processed_template_filename"]

# Output summary file
output_filename = params["output_filename"]

## Get unnormalized template experiment

In [6]:
process.fetch_template_experiment(
    normalized_compendium_filename,
    experiment_to_sample_filename,
    metadata_delimiter,
    experiment_id_colname,
    project_id,
    sample_id_colname,
    scaler_filename,
    raw_template_filename,
    normalized_template_filename
)

## Simulate data

In [7]:
# Simulate multiple experiments UPDATE COMMENT
# This step creates the following files in "<local_dir>/pseudo_experiment/" directory:
#   - selected_simulated_data_SRP012656_<n>.txt
#   - selected_simulated_encoded_data_SRP012656_<n>.txt
#   - template_normalized_data_SRP012656_test.txt
# in which "<n>" is an integer in the range of [0, num_runs-1]

# REMOVE LATER
NN_architecture = "NN_test"
dataset_name = "pre_model_seen_template"
# Load pickled file
scaler = pickle.load(open(scaler_filename, "rb"))

# Update simulated dir
os.makedirs(os.path.join(local_dir, "pseudo_experiment"), exist_ok=True)

#------------
simulate_expression_data.shift_template_experiment(
    normalized_compendium_filename,
    NN_architecture,
    latent_dim,
    dataset_name,
    scaler,
    experiment_to_sample_filename,
    metadata_delimiter,
    experiment_id_colname,
    sample_id_colname,
    project_id,
    local_dir,
    base_dir,
    num_runs,
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



## Process template and simulated experiments

* Remove samples not required for comparison
* Make sure ordering of samples matches metadata for proper comparison
* Make sure values are cast as integers if using DESeq
* Filter lowly expressed genes if using DESeq

In [8]:
# Update simulated dir
if not os.path.exists(template_process_samples_filename):
    template_process_samples_filename = None

if de_method == "deseq":
    # Process template data
    stats.process_samples_for_DESeq(
        raw_template_filename,
        template_DE_grouping_filename,
        processed_template_filename,
        count_threshold,
        template_process_samples_filename,
    )

    # Process simulated data
    for i in range(num_runs):
        simulated_filename = os.path.join(
            local_dir,
            "pseudo_experiment",
            f"selected_simulated_data_{project_id}_{i}.txt",
        )
        out_simulated_filename = os.path.join(
            local_dir,
            "pseudo_experiment",
            f"selected_simulated_data_{project_id}_{i}_processed.txt",
        )
        stats.process_samples_for_DESeq(
            simulated_filename,
            template_DE_grouping_filename,
            out_simulated_filename,
            count_threshold,
            template_process_samples_filename,
        )
else:
    stats.process_samples_for_limma(
        raw_template_filename,
        template_DE_grouping_filename,
        processed_template_filename,
        template_process_samples_filename,
    )

    for i in range(num_runs):
        simulated_filename = os.path.join(
            local_dir,
            "pseudo_experiment",
            f"selected_simulated_data_{project_id}_{i}.txt",
        )
        stats.process_samples_for_limma(
            simulated_filename,
            template_DE_grouping_filename,
            None,
            template_process_samples_filename,
        )

sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly


## Differential expression analysis

In [9]:
# Create subdirectory: "<local_dir>/DE_stats/"
os.makedirs(os.path.join(local_dir, "DE_stats"), exist_ok=True)

In [10]:
# Pass simulated dir to R scripts below

In [11]:
%%R -i template_DE_grouping_filename -i project_id -i processed_template_filename -i local_dir -i base_dir -i de_method

source(paste0(base_dir, '/sophie/DE_analysis.R'))

# File created: "<local_dir>/DE_stats/DE_stats_template_data_<project_id>_real.txt"
if (de_method == "deseq"){
    get_DE_stats_DESeq(
        template_DE_grouping_filename,
        project_id,
        processed_template_filename,
        "template",
        local_dir,
        "real"
    )
}
else{
    get_DE_stats_limma(
        template_DE_grouping_filename,
        project_id,
        processed_template_filename,
        "template",
        local_dir,
        "real"
    )
}

R[write to console]: Loading required package: S4Vectors

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following object is masked from ‘package:limma’:

    plotMA


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply

[1] "Checking sample ordering..."
[1] TRUE


R[write to console]: Error in !(all.equal(colnames(expression_data), metadata$sample)) : 
  invalid argument type

R[write to console]: In addition: 
R[write to console]: There were 12 warnings (use warnings() to see them)
R[write to console]: 




Error in !(all.equal(colnames(expression_data), metadata$sample)) : 
  invalid argument type


In [12]:
%%R -i template_DE_grouping_filename -i project_id -i base_dir -i local_dir -i num_runs -i de_method

source(paste0(base_dir, '/sophie/DE_analysis.R'))

# Files created: "<local_dir>/DE_stats/DE_stats_simulated_data_<project_id>_<n>.txt"
for (i in 0:(num_runs-1)){
    simulated_data_filename <- paste(
        local_dir,
        "pseudo_experiment/selected_simulated_data_",
        project_id,
        "_",
        i,
        "_processed.txt",
        sep = ""
    )
    if (de_method == "deseq"){
        get_DE_stats_DESeq(
            template_DE_grouping_filename,
            project_id,
            simulated_data_filename,
            "simulated",
            local_dir,
            i
            )
    }
    else {
        get_DE_stats_limma(
            template_DE_grouping_filename,
            project_id,
            simulated_data_filename,
            "simulated",
            local_dir,
            i
            )
        }
    }

[1] "Checking sample ordering..."
[1] TRUE


R[write to console]: Error in !(all.equal(colnames(expression_data), metadata$sample)) : 
  invalid argument type




Error in !(all.equal(colnames(expression_data), metadata$sample)) : 
  invalid argument type


## Rank genes

Genes are ranked by their "generic-ness" - how frequently these genes are changed across the simulated experiments using user-specific test statistic provided in the `col_to_rank_genes` params (i.e. log2 fold change).

In [13]:
analysis_type = "DE"
template_DE_stats_filename = os.path.join(
    local_dir, "DE_stats", f"DE_stats_template_data_{project_id}_real.txt"
)

# Added
if de_method == "deseq":
    logFC_name = "log2FoldChange"
    pvalue_name = "padj"
else:
    logFC_name = "logFC"
    pvalue_name = "adj.P.Val"

template_DE_stats, simulated_DE_summary_stats = ranking.process_and_rank_genes_pathways(
    template_DE_stats_filename,
    local_dir,
    num_runs,
    project_id,
    analysis_type,
    col_to_rank_genes,
    logFC_name,
    pvalue_name,
)

/home/alexandra/anaconda3/envs/sophie/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


## Summary table

* Gene ID: Gene identifier (hgnc symbols for human data or PA number for *P. aeruginosa* data)
* (Real): Statistics for template experiment
* (Simulated): Statistics across simulated experiments
* Number of experiments: Number of simulated experiments
* Z-score: High z-score indicates that gene is more changed in template compared to the null set of simulated experiments (high z-score = highly specific to template experiment)
* Percentile (simulated): percentile rank of the median(abs(log fold change)). So its the median absolute change for that gene across the 25 simulated experiments that is then converted to a percentile rank from 0 - 100. Where a higher percentile indicates that the gene was highly changed frequently and would suggest that the gene is more commonly DE.
* Percent DE (simulated): the fraction of the simulated experiments in which that gene was found to be DE using (log fold change > 1 and adjusted p-value < 0.05). _Note:_ you may find that many genes have a 0 fraction. This is because there is some compression that happens when pushing data through the VAE so the variance of the simulated experiments is lower compared to the real experiment. We are aware of this limitation in the VAE and are looking at how to improve the variance and biological signal captured by the VAE, however we were still able to demonstrate that for now the VAE is able to simulate realistic looking biological experiments in our previous [paper](https://academic.oup.com/gigascience/article/9/11/giaa117/5952607).


**Note:**
* If using DESeq, genes with NaN in only the `Adj P-value (Real)` column are those genes flagged because of the `cooksCutoff` parameter. The cook's distance as a diagnostic to tell if a single sample has a count which has a disproportionate impact on the log fold change and p-values. These genes are flagged with an NA in the pvalue and padj columns of the result table.

* If using DESeq with count threshold, some genes may not be present in all simulated experiments (i.e. the `Number of experiments (simulated)` will not equal the number of simulated experiments you specified in the beginning. This pre-filtering will lead to some genes found in few simulated experiments and so the background/null set for that gene is not robust. Thus, the user should sort by both z-score and number of experiments to identify specific expressed genes.

* If using DESeq without count threshold, some genes may still not be present in all simulated experiments (i.e. the `Number of experiments (simulated)`  will not equal the number of simulated experiments you specified in the beginning. If the gene is 0 expressed across all samples and thus automatically given an NA in `log fold change, adjusted p-value` columns. Thus, the user should sort by both z-score and number of experiments to identify specific expressed genes.

For more information you can read [DESeq FAQs](https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#pvaluesNA)

In [14]:
# Get summary table
summary_gene_ranks = ranking.generate_summary_table(
    template_DE_stats_filename,
    template_DE_stats,
    simulated_DE_summary_stats,
    col_to_rank_genes,
    local_dir,
    "gene",
    params,
)

summary_gene_ranks.sort_values(by="Z score", ascending=False).head(10)

,Gene ID,Adj P-value (Real),Rank (Real),abs(log2FoldChange) (Real),log2FoldChange (Real),Median adj p-value (simulated),Rank (simulated),Percentile (simulated),Percent DE (simulated),Mean abs(log2FoldChange) (simulated),Std deviation (simulated),Number of experiments (simulated),Z score
VEZT,VEZT,6.561899e-04,9622.0,0.564027,0.564027,0.045076,4234.0,23.842514,0.0,0.236518,0.000052,2,6301.055203
THAP9,THAP9,1.411167e-01,7741.0,0.399319,0.399319,0.784561,107.0,0.597049,0.0,0.027406,0.000059,2,6257.662855
TTC1,TTC1,2.812214e-02,8193.0,0.434710,0.434710,0.070044,1441.0,8.110848,0.0,0.115925,0.000118,2,2698.733956
LINC01730,LINC01730,1.301770e-02,13560.0,1.484307,1.484307,0.757420,303.0,1.701025,0.0,0.050050,0.000591,2,2428.170510
CIT,CIT,5.156598e-08,14996.0,2.899800,2.899800,0.254798,2050.0,11.541061,0.0,0.142667,0.001702,2,1619.890114
CASR,CASR,1.926148e-01,15482.0,4.649033,4.649033,0.402032,10815.0,60.910217,0.0,0.903815,0.002424,2,1544.822271
CWC15,CWC15,6.812821e-01,11302.0,0.788082,0.788082,0.003572,3123.0,17.584770,0.0,0.187333,0.000407,2,1474.512916
FAM110B,FAM110B,1.186656e-26,16007.0,21.693587,-21.693587,0.081285,9364.0,52.737411,0.0,0.614677,0.015747,2,1338.586384
DYNC1LI2,DYNC1LI2,4.908478e-02,5687.0,0.267402,0.267402,0.328713,882.0,4.962262,0.0,0.088109,0.000135,2,1324.782792
MUL1,MUL1,6.487886e-01,1893.0,0.069030,-0.069030,0.755263,56.0,0.309789,0.0,0.018416,0.000040,2,1266.050242


In [15]:
summary_gene_ranks.isna().any()

Gene ID                                 False
Adj P-value (Real)                       True
Rank (Real)                              True
abs(log2FoldChange) (Real)               True
log2FoldChange (Real)                    True
Median adj p-value (simulated)          False
Rank (simulated)                        False
Percentile (simulated)                  False
Percent DE (simulated)                  False
Mean abs(log2FoldChange) (simulated)    False
Std deviation (simulated)                True
Number of experiments (simulated)       False
Z score                                  True
dtype: bool

In [16]:
summary_gene_ranks[summary_gene_ranks.isna().any(axis=1)]

,Gene ID,Adj P-value (Real),Rank (Real),abs(log2FoldChange) (Real),log2FoldChange (Real),Median adj p-value (simulated),Rank (simulated),Percentile (simulated),Percent DE (simulated),Mean abs(log2FoldChange) (simulated),Std deviation (simulated),Number of experiments (simulated),Z score
A1CF,A1CF,NaN,NaN,NaN,NaN,0.000018,17140.0,96.535992,1.0,4.634457,3.969021,2,NaN
ABCB7,ABCB7,NaN,NaN,NaN,NaN,0.000667,14829.0,83.519207,0.5,2.869387,3.681591,2,NaN
ABCG5,ABCG5,NaN,NaN,NaN,NaN,0.000847,16269.0,91.630055,0.5,3.849066,4.605803,2,NaN
ABCG8,ABCG8,NaN,NaN,NaN,NaN,0.000624,15765.0,88.791258,0.5,3.507399,3.608295,2,NaN
ABRA,ABRA,NaN,NaN,NaN,NaN,0.002624,16148.0,90.948519,0.5,3.762359,4.433512,2,NaN
ACOD1,ACOD1,NaN,NaN,NaN,NaN,0.598339,12512.0,70.468627,0.0,1.595422,2.149427,2,NaN
ACRV1,ACRV1,NaN,NaN,NaN,NaN,0.342338,13511.0,76.095528,0.5,2.103757,2.895575,2,NaN
ACSM2A,ACSM2A,NaN,NaN,NaN,NaN,0.306849,12223.0,68.840825,0.0,1.451264,0.736960,2,NaN
ACTA1,ACTA1,NaN,NaN,NaN,NaN,0.046634,14490.0,81.609778,0.5,2.669649,2.018400,2,NaN
ACTC1,ACTC1,NaN,NaN,NaN,NaN,0.133748,14478.0,81.542188,0.5,2.661343,1.433808,2,NaN


In [17]:
# Save
summary_gene_ranks.to_csv(output_filename, sep="\t")